This is GANs, working on CGANs

> Add blockquote



In [2]:
import torch
import torch.nn as nn

In [3]:
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [1]:
import torch
print(torch.cuda.is_available())  # Should print True if GPU is available
print(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

True
cuda


In [4]:
%load_ext tensorboard

In [5]:
#discriminator uses conv2d as we need to do downsampling i.e. decrease the size and increase the number of features whereas
#generator usess ConvTranspose2d as we do unsampling i.e. increase the size and the number of features
class Discriminator(nn.Module):
  def __init__(self, channels_img, features_d):
    super(Discriminator, self).__init__()
    #the discriminator architecture
    self.disc=nn.Sequential(
        #input: NXchannels_imgX64x64
        nn.Conv2d(
            channels_img, features_d, kernel_size=4, stride=2, padding=1
        ),
        nn.LeakyReLU(0.2),
        self._block(features_d, features_d*2, 4, 2, 1),#16x16
        self._block(features_d*2, features_d*4, 4, 2, 1),#8x8
        self._block(features_d*4, features_d*8, 4, 2, 1),#4x4
        nn.Conv2d(features_d*8, 1, kernel_size=4, stride=2, padding=0),#1x1
        nn.Sigmoid(),
    )
    #generic block, to reduce redundancy in disc
  def _block(self, in_channels, out_channels, kernel_size, stride, padding):
    return nn.Sequential(
        nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            bias=False,
        ),
        nn.BatchNorm2d(out_channels),
        nn.LeakyReLU(0.2),
        )
    #defines how x flows through the discriminator
  def forward(self,x):
    return self.disc(x)

class Generator(nn.Module):
  def __init__(self, z_dim, channels_img, features_g):
    super(Generator, self).__init__()
    self.gen=nn.Sequential( #input Nxz_dimx1x1
        self._block(z_dim, features_g*16, 4, 1, 0),#Nxf_gx16x4x4
        self._block(features_g*16, features_g*8, 4, 2, 1),#8x8
        self._block(features_g*8, features_g*4, 4, 2, 1),#16x16
        self._block(features_g*4, features_g*2, 4, 2, 1),#32x32
        nn.ConvTranspose2d(
            features_g*2, channels_img, kernel_size=4, stride=2, padding=1
        ),
        nn.Tanh())
  def _block(self, in_channels, out_channels, kernel_size, stride, padding):
    return nn.Sequential(
        nn.ConvTranspose2d(
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            bias=False,
        ),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
    )
  def forward(self, x):
    return self.gen(x)

def initialize_weights(model):
  for m in model.modules():
    if isinstance(m,(nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
      nn.init.normal_(m.weight.data,0.0,0.02)
def test():
  N, in_channels, H,W= 8,3,64,64
  z_dim=100
  x=torch.randn((N, in_channels, H, W))
  disc= Discriminator(in_channels, 8)
  initialize_weights(disc)
  assert disc(x).shape==(N,1,1,1)
  gen=Generator(z_dim, in_channels, 8)
  initialize_weights(gen)
  z=torch.randn((N, z_dim, 1, 1))
  assert gen(z).shape==(N, in_channels, H, W)
  print( "success")


In [6]:
test()

success


In [7]:
#training model
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
LEARNING_RATE=2e-4
BATCH_SIZE=128
IMAGE_SIZE=64
CHANNELS_IMG=1
Z_DIM=100
EPOCH=5
FEATURES_DISC=64
FEATURES_GEN=64

image_transforms= transforms.Compose(
    [transforms.Resize(IMAGE_SIZE),
     transforms.ToTensor(),
     transforms.Normalize([0.5 for __ in range(CHANNELS_IMG)],[0.5 for __ in range(CHANNELS_IMG)]),]

)
dataset=datasets.MNIST(root="dataset/",train=True, transform=image_transforms,download=True)
loader= DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
gen = Generator(Z_DIM, CHANNELS_IMG, FEATURES_GEN).to(device)
disc= Discriminator(CHANNELS_IMG, FEATURES_DISC).to(device)
initialize_weights(gen)
initialize_weights(disc)
opt_gen=optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.5,0.999))
opt_disc=optim.Adam(disc.parameters(),lr=LEARNING_RATE, betas=(0.5,0.999))
criterion= nn.BCELoss()
fixed_noise=torch.randn(32,Z_DIM, 1,1).to(device)
writer_real=SummaryWriter(f"logs/real")
writer_fake=SummaryWriter(f"logs/fake")
step=0

gen.train()
disc.train()
for epoch in range(EPOCH):
  for batch_idx, (real,__) in enumerate(loader):
    real=real.to(device)
    noise=torch.randn((BATCH_SIZE, Z_DIM,1,1)).to(device)
    fake=gen(noise)
    disc_real=disc(real).reshape(-1)
    loss_disc_real=criterion(disc_real, torch.ones_like(disc_real))
    disc_fake=disc(fake).reshape(-1)
    loss_disc_fake=criterion(disc_fake, torch.zeros_like(disc_fake))
    loss_disc=(loss_disc_real+loss_disc_fake)/2
    disc.zero_grad()
    loss_disc.backward(retain_graph=True)
    opt_disc.step()

    output=disc(fake).reshape(-1)
    loss_gen=criterion(output, torch.ones_like(output))
    gen.zero_grad()
    loss_gen.backward()
    opt_gen.step()

    if batch_idx % 100 ==0:
      print(
          f"Epoch [{epoch}/{EPOCH}] Batch {batch_idx}/{len(loader)} \
          Loss D: {loss_disc:.4f}, loss G: {loss_gen:.4f}")
      with torch.no_grad():
        fake=gen(fixed_noise)
        img_grid_real=torchvision.utils.make_grid(real[:32],normalize=True)
        img_grid_fake=torchvision.utils.make_grid(fake[:32],normalize=True)

        writer_real.add_image("Real", img_grid_real, global_step=step)
        writer_fake.add_image("Fake", img_grid_fake, global_step=step)
      step+=1


Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:00<00:00, 47.9MB/s]


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 1.74MB/s]


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:00<00:00, 14.6MB/s]


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 8.44MB/s]


Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw

Epoch [0/5] Batch 0/469           Loss D: 0.6934, loss G: 0.7984
Epoch [0/5] Batch 100/469           Loss D: 0.0144, loss G: 4.1571
Epoch [0/5] Batch 200/469           Loss D: 0.6403, loss G: 0.8541
Epoch [0/5] Batch 300/469           Loss D: 0.5388, loss G: 1.2729
Epoch [0/5] Batch 400/469           Loss D: 0.4955, loss G: 1.2441
Epoch [1/5] Batch 0/469           Loss D: 0.5488, loss G: 0.7188
Epoch [1/5] Batch 100/469           Loss D: 0.6362, loss G: 0.7632
Epoch [1/5] Batch 200/469           Loss D: 0.6311, loss G: 0.9492
Epoch [1/5] Batch 300/469           Loss D: 0.5810, loss G: 1.4781
Epoch [1/5] Batch 400/469           Loss D: 0.5449, loss G: 1.0631
Epoch [2/5] Batch 0/469           Loss D: 0.5499, loss G: 0.8609
Epoch [2/5] Batch 100/469           Loss D: 0.5817, loss G: 0.6521
Epoch [2/5] Batch 200/469           Loss D: 0.5479, loss G: 1.3994
Epoch [2/5] Batch 300/469           Loss D: 0.5120, loss G

In [8]:
%tensorboard --logdir logs

Output hidden; open in https://colab.research.google.com to view.